<a href="https://colab.research.google.com/github/mtermor/NTIC_DeepLearning/blob/main/NLP/06_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Datasets

In [5]:
#!pip install datasets

In [7]:
from datasets import load_dataset

dataset = load_dataset('rotten_tomatoes')

Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

In [10]:
dataset['train'][0]

{'text': 'the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .',
 'label': 1}

In [11]:
dataset.__class__

datasets.dataset_dict.DatasetDict

In [12]:
dataset['train'][0].__class__

dict

In [13]:
from time import time
start = time()
dataset['train'][0]['text']
end = time()
print(f'>>>> Elaspsed time: {(end-start):4f}')

start = time()
dataset['train']['text'][0]
end = time()
print(f'>>>> Elaspsed time: {(end-start):4f}')

>>>> Elaspsed time: 0.002203
>>>> Elaspsed time: 0.012688


In [14]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [15]:
tokenizer(dataset['train'][0]['text'])

{'input_ids': [101, 1996, 2600, 2003, 16036, 2000, 2022, 1996, 7398, 2301, 1005, 1055, 2047, 1000, 16608, 1000, 1998, 2008, 2002, 1005, 1055, 2183, 2000, 2191, 1037, 17624, 2130, 3618, 2084, 7779, 29058, 8625, 13327, 1010, 3744, 1011, 18856, 19513, 3158, 5477, 4168, 2030, 7112, 16562, 2140, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [16]:
def tokenization(x):
  return tokenizer(x['text'])

In [19]:
# dataset = dataset.map(tokenization)
# dataset = dataset.map(tokenization, batched=True)
dataset = dataset.map(tokenization, batch_size = 10000)

Map:   0%|          | 0/8530 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

In [20]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1066
    })
})

In [22]:
# dataset['train'][0]

## Metrics

In [24]:
from datasets import list_metrics

# list_metrics()

## Creación de dataset

In [30]:
from datasets import Dataset, DatasetDict

ds = Dataset.from_dict({
    'pokemon':['pikatchu','bulbasour'],
    'type': ['grass','water']
})

ds

Dataset({
    features: ['pokemon', 'type'],
    num_rows: 2
})

# Fine Tuning: Ejemplo (Clasificación)

In [1]:
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/eduardofc/data/main/amazon_sports.csv')
df['review_body'] = df['review_body'].str.replace('[^a-zA-ZñÑáéíóú .,]','', regex=True)
df['review_body'] = df['review_body'].str.lower()
df.head()

,stars,review_body,review_title,product_category
0,1,nunca llego el pedido y el vendedor pasa de to...,No llego nunca,sports
1,1,"no sé como es, porque debería haber llegado ay...",Todavía no ha llegado,sports
2,1,"guantes cómodos, no lo niego, pero de mala cal...",Guantes de baja calidad,sports
3,1,hasta hoy no he visto el producto. el pedido h...,Muy Mala experiencia,sports
4,1,"no puedo valorarla porque, después de casi una...",Paquete perdido?,sports


In [2]:
df = df[df.stars != 2]
df = df[df.stars != 3]
df = df[df.stars != 4]
df['good_product'] = (df.stars>3).astype(int)
df.groupby('good_product').size()

good_product
0    2438
1    2512
dtype: int64

In [3]:
from sklearn.model_selection import train_test_split
X = df['review_body'].values
y = df['good_product']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=99)

df_train = pd.DataFrame(zip(X_train, y_train), columns=['review_body', 'labels'])
df_test = pd.DataFrame(zip(X_test, y_test), columns=['review_body', 'labels'])

In [4]:
#!pip install datasets

In [5]:
from datasets import Dataset, DatasetDict

ds_train = Dataset.from_pandas(df_train)
ds_test = Dataset.from_pandas(df_test)

ds = DatasetDict()
ds['train'] = ds_train
ds['test'] = ds_test

ds

DatasetDict({
    train: Dataset({
        features: ['review_body', 'labels'],
        num_rows: 3960
    })
    test: Dataset({
        features: ['review_body', 'labels'],
        num_rows: 990
    })
})

In [6]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "distilbert/distilbert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
model.__class__

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification

In [7]:
def tokenize_function(x):
  return tokenizer(x['review_body'], truncation=True)

tokenized_ds = ds.map(tokenize_function, batched=True)

Map:   0%|          | 0/3960 [00:00<?, ? examples/s]

Map:   0%|          | 0/990 [00:00<?, ? examples/s]

In [8]:
tokenized_ds

DatasetDict({
    train: Dataset({
        features: ['review_body', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 3960
    })
    test: Dataset({
        features: ['review_body', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 990
    })
})

## Training

In [9]:
#!pip install transformers[torch]
#!pip install accelerate -U

In [10]:
from transformers import TrainingArguments

training_args = TrainingArguments(model_name)
# training_args

In [11]:
training_args = TrainingArguments(
    output_dir = './results_ejemplo',
    num_train_epochs = 1,
    per_device_train_batch_size = 1,
    do_train=True
)
# training_args

In [12]:
# DataCollator
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer = tokenizer)

In [13]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset = tokenized_ds['train'],
    tokenizer = tokenizer
)

In [14]:
trainer.train()

Step,Training Loss
500,0.902700
1000,1.073800
1500,0.784200
2000,0.620400
2500,0.629900
3000,0.539500
3500,0.486100


TrainOutput(global_step=3960, training_loss=0.6921986319802024, metrics={'train_runtime': 285.5184, 'train_samples_per_second': 13.87, 'train_steps_per_second': 13.87, 'total_flos': 52493570147484.0, 'train_loss': 0.6921986319802024, 'epoch': 1.0})

## Evaluation

In [15]:
predictions = trainer.predict(tokenized_ds['test'])

In [16]:
# predictions

PredictionOutput(predictions=array([[ 3.5893052, -3.0904384],
       [-3.592437 ,  4.0781603],
       [ 2.9019   , -2.4371815],
       ...,
       [-3.6774657,  4.181434 ],
       [ 3.450444 , -2.9382095],
       [ 3.50856  , -3.004284 ]], dtype=float32), label_ids=array([0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1,
       0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0,
       0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0,
       1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0,
       1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0,
       1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0,
       1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0,
    

In [17]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

In [18]:
preds = np.argmax(predictions.predictions, axis = -1)

In [19]:
accuracy_score(y_pred = preds, y_true = tokenized_ds['test']['labels'])

0.907070707070707

In [20]:
f1_score(y_pred = preds, y_true = tokenized_ds['test']['labels'])

0.9094488188976377